# T1110.002 - Password Cracking
Adversaries may use password cracking to attempt to recover usable credentials, such as plaintext passwords, when credential material such as password hashes are obtained. [OS Credential Dumping](https://attack.mitre.org/techniques/T1003) can be used to obtain password hashes, this may only get an adversary so far when [Pass the Hash](https://attack.mitre.org/techniques/T1550/002) is not an option. Further,  adversaries may leverage [Data from Configuration Repository](https://attack.mitre.org/techniques/T1602) in order to obtain hashed credentials for network devices.(Citation: US-CERT-TA18-106A) 

Techniques to systematically guess the passwords used to compute hashes are available, or the adversary may use a pre-computed rainbow table to crack hashes. Cracking hashes is usually done on adversary-controlled systems outside of the target network.(Citation: Wikipedia Password cracking) The resulting plaintext password resulting from a successfully cracked hash may be used to log into systems, resources, and services in which the account has access.

## Atomic Tests

### Atomic Test #1 - Password Cracking with Hashcat
Execute Hashcat.exe with provided SAM file from registry of Windows and Password list to crack against
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Hashcat must exist on disk at specified location (#{hashcat_exe})
##### Check Prereq Commands:
```cmd
if (Test-Path  $(cmd /c echo %temp%\hashcat6\hashcat-6.1.1\hashcat.exe)) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://www.7-zip.org/a/7z1900.exe" -OutFile "$env:TEMP\7z1900.exe"
Start-Process -FilePath "$env:Temp\7z1900.exe" -ArgumentList "/S /D=$env:temp\7zi" -NoNewWindow
Invoke-WebRequest "https://hashcat.net/files/hashcat-6.1.1.7z" -OutFile "$env:TEMP\hashcat6.7z"
Start-Process cmd.exe -Args  "/c %temp%\7z\7z.exe x %temp%\hashcat6.7z -aoa -o%temp%\hashcat-unzip" -Wait
New-Item -ItemType Directory (Split-Path $(cmd /c echo %temp%\hashcat6\hashcat-6.1.1\hashcat.exe)) -Force | Out-Null
Move-Item $env:Temp\hashcat-unzip\hashcat-6.1.1\* $(cmd /c echo %temp%\hashcat6\hashcat-6.1.1\hashcat.exe\..) -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1110.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cd %temp%\hashcat6\hashcat-6.1.1\hashcat.exe\..
%temp%\hashcat6\hashcat-6.1.1\hashcat.exe -a 0 -m 1000 -r .\rules\Incisive-leetspeak.rule PathToAtomicsFolder\T1110.002\src\sam.txt PathToAtomicsFolder\T1110.002\src\password.lst```

In [ ]:
Invoke-AtomicTest T1110.002 -TestNumbers 1

#### Cleanup: 
```cmd
del %temp%\hashcat6.7z >nul 2>&1
del %temp%\7z1900.exe >nul 2>&1
del %temp%\7z /Q /S >nul 2>&1
del %temp%\hashcat-unzip /Q /S >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1110.002 -TestNumbers 1 -Cleanup

## Detection
It is difficult to detect when hashes are cracked, since this is generally done outside the scope of the target network. Consider focusing efforts on detecting other adversary behavior used to acquire credential materials, such as [OS Credential Dumping](https://attack.mitre.org/techniques/T1003) or [Kerberoasting](https://attack.mitre.org/techniques/T1558/003).